# Bayesian inference examples

https://inferpy.readthedocs.io/projects/develop/en/latest/notes/probzoo.html



## Probabilistic models
https://inferpy.readthedocs.io/projects/develop/en/latest/notes/guidemodels.html#probabilistic-models
    

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import inferpy as inf


import plotly.express as px
import plotly.graph_objects as go


## Bayesian Linear Regression

https://inferpy.readthedocs.io/projects/develop/en/latest/notes/probzoo.html#bayesian-linear-regression

## Probabilistic model

In [26]:
# Linear regression model (used for data generation and modelling)

@inf.probmodel
def linear_reg(d):
    w0 = inf.Normal(0, 1, name="w0")
    w = inf.Normal(np.zeros([d, 1]), 1, name="w")

    with inf.datamodel():
        #x = inf.Normal(tf.ones(d), 2, name="x")
        x = inf.Normal(tf.ones(d), 1.0, name="x")
        y = inf.Normal(w0 + x @ w, 1.0, name="y")






In [27]:


@inf.probmodel
def qmodel(d):
    qw0_loc = inf.Parameter(14., name="qw0_loc")
    qw0_scale = tf.math.softplus(inf.Parameter(1., name="qw0_scale"))
    qw0 = inf.Normal(qw0_loc, qw0_scale, name="w0")

    qw_loc = inf.Parameter(np.zeros([d, 1]), name="qw_loc")
    qw_scale = tf.math.softplus(inf.Parameter(tf.ones([d, 1]), name="qw_scale"))
    qw = inf.Normal(qw_loc, qw_scale, name="w")


# Generic components

Exploring Bayesian inference models

In [28]:
from sklearn.linear_model import LinearRegression


def generate_data(n, w0, w):

    m = linear_reg(d=len(w))
    data = m.prior(["x", "y"], data={"w0": w0, "w": w}, size_datamodel=n).sample()

    x_train = data["x"]
    y_train = data["y"]

    for i in range(len(w)):

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x_train[:, i], y=[_[0] for _ in y_train], mode='markers'))
        fig.update_layout(
            title=f"ix:{i} Generated data y = {w0} + {w}x"
        )
        fig.show()
    
    
    
    return data


def train_linear_regression(x_train, y_train):

    reg = LinearRegression().fit(X=x_train, y=y_train)
    reg.get_params()
    print(f"Linear regression model:  intercept: {reg.intercept_}  gradient: {reg.coef_}")

    return reg

In [29]:
def train_bayes_inference(x_train, y_train, d):

    m = linear_reg(d=d)
    q = qmodel(d)

    VI = inf.inference.VI(q, epochs=10000)
    m.fit({"x": x_train, "y": y_train}, VI)

    # extract the parameters of the posterior
    res = m.posterior(["w", "w0"]).parameters()
    
    print(f"Bayesian inference model:  intercept: {res['w0']['loc']}  gradient: {res['w']['loc']}") 
    
    return m, res

# Model with d=2

* We generate some data according to a simple linear regression model with 2 features.
* We fit a linear regression model to get intercept and weight parameters 
* We fit a Bayesian inference model and inspect the model parameters


## What do we expect?

* Linear regression parameters are close to those from the model used to generate the data 
* Bayes inference model also has parameters close to those from the model used to generate the data 

## What do we observe?

* Linear regression parameters ARE close to those used to generate the data
* Bayesian inference model regression parameters ARE NOT close to those used to generate the data !

## Explanation

Over to you Prerit Shah! :-)



In [30]:
# Model with d=2

W0 = 20
W = [[2],[3]]

data = generate_data(1000, W0, W)

x_train = data["x"]
y_train = data["y"]


model_lin_reg = train_linear_regression(x_train, y_train)
model_bayes = train_bayes_inference(x_train, y_train, len(W))

Linear regression model:  intercept: [20.017021]  gradient: [[2.0113714 3.009615 ]]

 0 epochs	 22819.947265625....................
 200 epochs	 49697.65234375....................
 400 epochs	 73711.9921875....................
 600 epochs	 46985.2578125....................
 800 epochs	 64611.8984375....................
 1000 epochs	 60005.51171875....................
 1200 epochs	 42741.6015625....................
 1400 epochs	 35259.59375....................
 1600 epochs	 33858.7890625....................
 1800 epochs	 18325.6328125....................
 2000 epochs	 22465.75390625....................
 2200 epochs	 11768.30078125....................
 2400 epochs	 10320.1962890625....................
 2600 epochs	 31091.056640625....................
 2800 epochs	 19875.22265625....................
 3000 epochs	 17108.099609375....................
 3200 epochs	 8893.109375....................
 3400 epochs	 8258.4453125....................
 3600 epochs	 10847.4521484375...................

## d=2: some old results

Compare the coefficients yielded by a linear regression fit reg.intercept_ and reg.coef_
with those yielded by Bayes inference res['w0']['loc'] and res['w']['loc']


| Input w0  | w  | LinReg w0 | w   | Bayes w0 | w    | comments |
| -------| ------|---------- | ----|--------  |------|----------|
| 1      | 2, 3  | 1         | 2,3 | 1        | 2,3  | same as example on inferpy |
| 5      | 2, 3  | 5         | 2,3 | 5        | 2,3  |  |
| 20      | 2, 3 | 20        | 2,3 | 9        | 4,5  | after 10k epochs |
| 20      | 2, 3 | 20        | 2,3 | 17       | 2.5,3.4  | after 20k epochs |
| 20      | 2, 3 | 20        | 2,3 | 20       | 2,3  | after 30k epochs |


Conclusion: it's important to look at learning performance / convergence.  

## model with d=1

In [14]:
# Model with d=1

W0 = 20
W = [[2]]

data = generate_data(1000, W0, W)

x_train = data["x"]
y_train = data["y"]

model_lin_reg = train_linear_regression(x_train, y_train)
model_bayes = train_bayes_inference(x_train, y_train, len(W))

Linear regression model:  intercept: [20.012018]  gradient: [[2.0182657]]

 0 epochs	 243982.1875....................
 200 epochs	 328554.53125....................
 400 epochs	 217790.390625....................
 600 epochs	 252402.21875....................
 800 epochs	 182635.390625....................
 1000 epochs	 207675.3125....................
 1200 epochs	 191599.09375....................
 1400 epochs	 145204.015625....................
 1600 epochs	 179836.359375....................
 1800 epochs	 176129.921875....................
 2000 epochs	 185582.015625....................
 2200 epochs	 168053.796875....................
 2400 epochs	 137279.234375....................
 2600 epochs	 187159.6875....................
 2800 epochs	 162248.265625....................
 3000 epochs	 182879.03125....................
 3200 epochs	 138974.25....................
 3400 epochs	 122517.4921875....................
 3600 epochs	 154933.25....................
 3800 epochs	 105969.015625..........

## d=1: some old results

Compare the coefficients yielded by a linear regression fit reg.intercept_ and reg.coef_
with those yielded by Bayes inference res['w0']['loc'] and res['w']['loc']


| Input w0  | w  | LinReg w0 | w   | Bayes w0 | w    | comments |
| -------| ------|---------- | ----|--------  |------|----------|
| 0      | 2     | 0         | 2   | 0        | 2    | after 30k epochs |
| 0      | 2     | 0         | 2   | 0        | 2    | after 10k epochs |
| 5      | 2     | 5         | 2   | 5        | 2    | after 10k epochs |
| 20      | 2    | 20         | 2   | 5        | 2    | after 10k epochs |





----------------------------------------------------

In [7]:
# Single-cell example from the inferpy tutorials
#https://inferpy.readthedocs.io/projects/develop/en/latest/notes/probzoo.html#bayesian-linear-regression


import inferpy as inf
import tensorflow as tf
import numpy as np

@inf.probmodel
def linear_reg(d):
    w0 = inf.Normal(0, 1, name="w0")
    w = inf.Normal(np.zeros([d, 1]), 1, name="w")

    with inf.datamodel():
        x = inf.Normal(tf.ones(d), 2, name="x")
        y = inf.Normal(w0 + x @ w, 1.0, name="y")


@inf.probmodel
def qmodel(d):
    qw0_loc = inf.Parameter(0., name="qw0_loc")
    qw0_scale = tf.math.softplus(inf.Parameter(1., name="qw0_scale"))
    qw0 = inf.Normal(qw0_loc, qw0_scale, name="w0")

    qw_loc = inf.Parameter(np.zeros([d, 1]), name="qw_loc")
    qw_scale = tf.math.softplus(inf.Parameter(tf.ones([d, 1]), name="qw_scale"))
    qw = inf.Normal(qw_loc, qw_scale, name="w")


# create an instance of the model
m = linear_reg(d=2)
q = qmodel(2)
# create toy data
N = 1000
data = m.prior(["x", "y"], data={"w0": 0, "w": [[2], [1]]}, size_datamodel=N).sample()

x_train = data["x"]
y_train = data["y"]

# set and run the inference
VI = inf.inference.VI(qmodel(2), epochs=10000)
m.fit({"x": x_train, "y": y_train}, VI)


# extract the parameters of the posterior
m.posterior(["w", "w0"]).parameters()



 0 epochs	 16139.8974609375....................
 200 epochs	 25217.064453125....................
 400 epochs	 34219.3125....................
 600 epochs	 14437.5966796875....................
 800 epochs	 12137.1591796875....................
 1000 epochs	 8311.685546875....................
 1200 epochs	 6376.38916015625....................
 1400 epochs	 10270.3671875....................
 1600 epochs	 20380.208984375....................
 1800 epochs	 11312.21484375....................
 2000 epochs	 12309.7216796875....................
 2200 epochs	 12881.654296875....................
 2400 epochs	 13073.548828125....................
 2600 epochs	 9498.2685546875....................
 2800 epochs	 6307.79052734375....................
 3000 epochs	 6055.9150390625....................
 3200 epochs	 11991.8671875....................
 3400 epochs	 7235.00537109375....................
 3600 epochs	 8101.52685546875....................
 3800 epochs	 6155.9052734375....................
 4000 epo

{'w0': {'loc': -0.054548524,
  'scale': 0.16256933,
  'validate_args': False,
  'allow_nan_stats': True,
  'name': 'w0'},
 'w': {'loc': array([[2.0002897],
         [1.0216105]], dtype=float32),
  'scale': array([[0.12546477],
         [0.11237253]], dtype=float32),
  'validate_args': False,
  'allow_nan_stats': True,
  'name': 'w'}}